# Merge autoplus

In [3]:
import pandas as pd
import fuzzy_pandas as fpd

In [14]:
autoplus = pd.read_csv('autoplus.csv')
dcar = pd.read_csv('Data_cars.csv')

In [15]:
autoplus = autoplus.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'])

In [16]:
autoplus.head()

,BOITE_VITESSE,DATE,ENERGIE,MALUS,MARQUE_MODELE,PRIX_NEUF,PUISS_FISCALE,VERSION
0,Automatique,04/2013,Électrique,n.a.,Renault Twizy,7540,1CV,Electrique 45 BVA Life (2p.)
1,Manuelle,06/2019,Essence,100,Dacia Logan,8150,4CV,1.0 12V Sce 75 Access (4p.)
2,Automatique,04/2013,Électrique,n.a.,Renault Twizy,8240,2CV,Electrique 80 BVA Life (2p.)
3,Manuelle,06/2019,Essence,100,Dacia Sandero,8290,4CV,1.0i SCe 75 Access (5p.)
4,Automatique,07/2015,Électrique,n.a.,Renault Twizy,8340,1CV,Electrique 45 BVA Intens Blanc (2p.)


In [17]:
dcar.head()

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI ..."
1,57526.0,BMW,X6,2015,39051.0 km,Diesel,automatique,03/04/2018 à 16h41,"modele: X6 F16, version: (F16) XDRIVE30D 258 M..."
2,80379.0,AUDI,RS6,2014,75381.0 km,Essence,automatique,30/07/2018 à 1h55,"modele: RS6 (3E GENERATION) AVANT, version: II..."
3,2830.0,FORD,KA,2007,92282.0 km,Essence,mécanique,09/07/2018 à 14h12,"modele: KA, version: 1.3 70 FUN, puissance_fis..."
4,12925.0,FIAT,TIPO,2018,10.0 km,Essence,mécanique,19/05/2018 à 3h52,"modele: TIPO 2 SW, version: II SW 1.4 95 POP, ..."


In [19]:
dcar['MARQUE_MODELE'] = dcar['Make'] + ' ' + dcar['Model']

In [20]:
matches = fpd.fuzzy_merge(autoplus, dcar,
                          left_on=['MARQUE_MODELE'],
                          right_on=['MARQUE_MODELE'],
                          ignore_case=True,
                          keep='match')

print(matches)


KeyboardInterrupt: 